In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import logging

# 모든 로그의 기본 레벨을 WARNING 이상으로 설정
logging.basicConfig(level=logging.WARNING)

# 개별 모듈의 로그 레벨 조정
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)

In [3]:
%pip install prophet

# data 전처리

In [4]:
import pandas as pd
from prophet import Prophet

df=pd.read_csv('/content/gdrive/MyDrive/EURON 프로젝트/train.csv')

In [5]:
df = df.rename(columns={'일시':'date','광진구': 'G','동대문구': 'D','성동구': 'S','중랑구': 'J'})

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['is_weekend'] = df['weekday'].isin([5,6])

In [7]:
import holidays

kr_holidays = holidays.KR(years=range(2018, 2023))

df['holiday'] = df['date'].apply(lambda x: kr_holidays.get(x) if x in kr_holidays else 'not_holiday')
df['is_holiday'] = df['holiday'] != 'not_holiday'

df.head()

,date,G,D,S,J,year,month,day,weekday,is_weekend,holiday,is_holiday
0,2018-01-01,0.592,0.368,0.580,0.162,2018,1,1,0,False,New Year's Day,True
1,2018-01-02,0.840,0.614,1.034,0.260,2018,1,2,1,False,not_holiday,False
2,2018-01-03,0.828,0.576,0.952,0.288,2018,1,3,2,False,not_holiday,False
3,2018-01-04,0.792,0.542,0.914,0.292,2018,1,4,3,False,not_holiday,False
4,2018-01-05,0.818,0.602,0.994,0.308,2018,1,5,4,False,not_holiday,False


# holidays 파라미터 생성

In [8]:
holidays = df[df['is_holiday']].copy()
holidays = holidays[['date', 'holiday']].rename(columns={'date': 'ds'})

#광진구

In [9]:
df_G= pd.DataFrame({'ds': df['date'],'y': df['G']})

df_G.head()

,ds,y
0,2018-01-01,0.592
1,2018-01-02,0.840
2,2018-01-03,0.828
3,2018-01-04,0.792
4,2018-01-05,0.818


In [10]:
best_params = {
    'changepoint_prior_scale': 0.1,
    'daily_seasonality': 10,
    'seasonality_prior_scale': 15,
    'weekly_seasonality': 10,
    'yearly_seasonality': 2
}

In [11]:
import random
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

train_G=df_G[df_G['ds']<'2021-01-01']
val_G=df_G[df_G['ds']>='2021-01-01']


model_G= Prophet(
        changepoint_prior_scale =0.1,
        seasonality_mode = 'multiplicative',
        seasonality_prior_scale = 15,
        weekly_seasonality=10,
        daily_seasonality =10,
        yearly_seasonality =2,
        interval_width=0.95,holidays=holidays )

model_G.fit(train_G)

future_G= model_G.make_future_dataframe(periods = 365, freq='D', include_history=False)
forecast_G = model_G.predict(future_G)


y_true_G = val_G['y'].values
y_pred_G = forecast_G['yhat'].abs().values

mae = mean_absolute_error(y_true_G, y_pred_G)
rmse = mean_squared_error(y_true_G, y_pred_G)

print(f"MAE = {mae:.3f} | RMSE = {rmse:.3f}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/wcnqluq7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/1d94_zg8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=74893', 'data', 'file=/tmp/tmp5qdiny0e/wcnqluq7.json', 'init=/tmp/tmp5qdiny0e/1d94_zg8.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modelxmr4t9jf/prophet_model-20250716152639.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


MAE = 1.865 | RMSE = 6.991


## 2022 예측 (2018-2021 재학습)

In [12]:
train_G_full = df_G[df_G['ds'] < '2022-01-01']

model_G_final = Prophet(
    changepoint_prior_scale=0.1,
    seasonality_mode='multiplicative',
    seasonality_prior_scale=15,
    weekly_seasonality=10,
    daily_seasonality=10,
    yearly_seasonality=2,
    interval_width=0.95,
    holidays=holidays
)

model_G_final.fit(train_G_full)

# 2021 + 2022년 11개월 예측 기간 (총 699일)
future_G_final = model_G_final.make_future_dataframe(periods=699, freq='D', include_history=False)
forecast_G_final = model_G_final.predict(future_G_final)
forecast_G_final['yhat'] = np.where(forecast_G_final['yhat'] < 0, 0, forecast_G_final['yhat'])

# 2022년 1월 1일부터 11월 30일까지 필터링
forecast_G_2022 = forecast_G_final.loc[
    forecast_G_final['ds'].between('2022-01-01', '2022-11-30'),
    ['ds', 'yhat']
].reset_index(drop=True)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/chnwms3r.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/uztvdc6z.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86193', 'data', 'file=/tmp/tmp5qdiny0e/chnwms3r.json', 'init=/tmp/tmp5qdiny0e/uztvdc6z.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modeldq964ssb/prophet_model-20250716152642.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:43 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


# 동대문구

In [13]:
df_D= pd.DataFrame({'ds': df['date'],'y': df['D']})

df_D.head()

,ds,y
0,2018-01-01,0.368
1,2018-01-02,0.614
2,2018-01-03,0.576
3,2018-01-04,0.542
4,2018-01-05,0.602


In [14]:
best_params = {
    'changepoint_prior_scale': 0.2,
    'seasonality_prior_scale': 10,
    'weekly_seasonality': 20,
    'daily_seasonality': 15,
    'yearly_seasonality': 5
}

In [15]:
import random
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

train_D=df_D[df_D['ds']<'2021-01-01']
val_D=df_D[df_D['ds']>='2021-01-01']


model_D= Prophet(
        changepoint_prior_scale =0.2,
        seasonality_mode = 'multiplicative',
        seasonality_prior_scale =10,
        weekly_seasonality=20,
        daily_seasonality =15,
        yearly_seasonality =5,
        interval_width=0.95,holidays=holidays )

model_D.fit(train_D)

future_D= model_D.make_future_dataframe(periods = 365, freq='D', include_history=False)
forecast_D = model_D.predict(future_D)


y_true_D= val_D['y'].values
y_pred_D = forecast_D['yhat'].abs().values

mae = mean_absolute_error(y_true_D, y_pred_D)
rmse = mean_squared_error(y_true_D, y_pred_D)

print(f"MAE = {mae:.3f} | RMSE = {rmse:.3f}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/95d3mvu7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/9iqdcs6x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=13243', 'data', 'file=/tmp/tmp5qdiny0e/95d3mvu7.json', 'init=/tmp/tmp5qdiny0e/9iqdcs6x.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modelz1smemea/prophet_model-20250716152646.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


MAE = 1.524 | RMSE = 4.340


## 2022 예측 (2018-2021 재학습)

In [16]:
train_D_full = df_D[df_D['ds'] < '2022-01-01']

model_D_final = Prophet(
    changepoint_prior_scale=0.1,
    seasonality_mode='multiplicative',
    seasonality_prior_scale=15,
    weekly_seasonality=10,
    daily_seasonality=10,
    yearly_seasonality=2,
    interval_width=0.95,
    holidays=holidays
)

model_D_final.fit(train_D_full)

future_D_final = model_D_final.make_future_dataframe(periods=699, freq='D', include_history=False)
forecast_D_final = model_D_final.predict(future_D_final)
forecast_D_final['yhat'] = np.where(forecast_D_final['yhat'] < 0, 0, forecast_D_final['yhat'])

forecast_D_2022 = forecast_D_final.loc[
    forecast_D_final['ds'].between('2022-01-01', '2022-11-30'),
    ['ds', 'yhat']
].reset_index(drop=True)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/dw2vhpx8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/obg3votc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38938', 'data', 'file=/tmp/tmp5qdiny0e/dw2vhpx8.json', 'init=/tmp/tmp5qdiny0e/obg3votc.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modela7kcqcp9/prophet_model-20250716152650.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


#성동구

In [17]:
df_S= pd.DataFrame({'ds': df['date'],'y': df['S']})

df_S.head()

,ds,y
0,2018-01-01,0.580
1,2018-01-02,1.034
2,2018-01-03,0.952
3,2018-01-04,0.914
4,2018-01-05,0.994


In [18]:
best_params = {
    'changepoint_prior_scale': 0.1,
    'seasonality_prior_scale': 5,
    'weekly_seasonality': 20,
    'daily_seasonality': 10,
    'yearly_seasonality': 5
}

In [19]:
import random
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

train_S=df_S[df_S['ds']<'2021-01-01']
val_S=df_S[df_S['ds']>='2021-01-01']


model_S= Prophet(
        changepoint_prior_scale =0.1,
        seasonality_mode = 'multiplicative',
        seasonality_prior_scale = 5,
        weekly_seasonality=20,
        daily_seasonality =10,
        yearly_seasonality=5,
        interval_width=0.95,holidays=holidays )

model_S.fit(train_S)

future_S= model_S.make_future_dataframe(periods = 365, freq='D', include_history=False)
forecast_S = model_S.predict(future_S)


y_true_S= val_S['y'].values
y_pred_S = forecast_S['yhat'].abs().values

mae = mean_absolute_error(y_true_S, y_pred_S)
rmse = mean_squared_error(y_true_S, y_pred_S)

print(f"MAE = {mae:.3f} | RMSE = {rmse:.3f}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/3qpj0l2a.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/xcoodtje.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=90735', 'data', 'file=/tmp/tmp5qdiny0e/3qpj0l2a.json', 'init=/tmp/tmp5qdiny0e/xcoodtje.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modelbsgbq79a/prophet_model-20250716152651.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


MAE = 1.745 | RMSE = 6.082


##2022 예측(2018-2021 재학습)

In [20]:
train_S_full = df_S[df_S['ds'] < '2022-01-01']

model_S_final = Prophet(
    changepoint_prior_scale=0.1,
    seasonality_mode='multiplicative',
    seasonality_prior_scale=15,
    weekly_seasonality=10,
    daily_seasonality=10,
    yearly_seasonality=2,
    interval_width=0.95,
    holidays=holidays
)

model_S_final.fit(train_S_full)

future_S_final = model_S_final.make_future_dataframe(periods=699, freq='D', include_history=False)
forecast_S_final = model_S_final.predict(future_S_final)
forecast_S_final['yhat'] = np.where(forecast_S_final['yhat'] < 0, 0, forecast_S_final['yhat'])

forecast_S_2022 = forecast_S_final.loc[
    forecast_S_final['ds'].between('2022-01-01', '2022-11-30'),
    ['ds', 'yhat']
].reset_index(drop=True)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/ysqa_kd1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/36wdc9lx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=31722', 'data', 'file=/tmp/tmp5qdiny0e/ysqa_kd1.json', 'init=/tmp/tmp5qdiny0e/36wdc9lx.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modeldxvink5x/prophet_model-20250716152652.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:52 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:53 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


#중랑구

In [21]:
df_J= pd.DataFrame({'ds': df['date'],'y': df['J']})

df_J.head()

,ds,y
0,2018-01-01,0.162
1,2018-01-02,0.260
2,2018-01-03,0.288
3,2018-01-04,0.292
4,2018-01-05,0.308


In [22]:
best_params = {
    'changepoint_prior_scale': 0.1,
    'seasonality_prior_scale': 5,
    'weekly_seasonality': 15,
    'daily_seasonality': 10,
    'yearly_seasonality': 2
}

In [23]:
import random
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

train_J=df_J[df_J['ds']<'2021-01-01']
val_J=df_J[df_J['ds']>='2021-01-01']


model_J= Prophet(
        changepoint_prior_scale =0.1,
        seasonality_mode = 'multiplicative',
        seasonality_prior_scale =5,
        weekly_seasonality=15,
        daily_seasonality =10,
        yearly_seasonality =2,
        interval_width=0.95,holidays=holidays )

model_J.fit(train_J)

future_J= model_J.make_future_dataframe(periods = 365, freq='D', include_history=False)
forecast_J = model_J.predict(future_J)


y_true_J = val_J['y'].values
y_pred_J = forecast_J['yhat'].abs().values

mae = mean_absolute_error(y_true_J, y_pred_J)
rmse = mean_squared_error(y_true_J, y_pred_J)

print(f"MAE = {mae:.3f} | RMSE = {rmse:.3f}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/kb6pdkt2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/1ivzxf74.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=88042', 'data', 'file=/tmp/tmp5qdiny0e/kb6pdkt2.json', 'init=/tmp/tmp5qdiny0e/1ivzxf74.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modelopiiw5fp/prophet_model-20250716152654.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


MAE = 1.057 | RMSE = 1.967


## 2022 예측 (2018-2021 재학습)

In [24]:
train_J_full = df_J[df_J['ds'] < '2022-01-01']

model_J_final = Prophet(
    changepoint_prior_scale=0.1,
    seasonality_mode='multiplicative',
    seasonality_prior_scale=15,
    weekly_seasonality=10,
    daily_seasonality=10,
    yearly_seasonality=2,
    interval_width=0.95,
    holidays=holidays
)

model_J_final.fit(train_J_full)

future_J_final = model_J_final.make_future_dataframe(periods=699, freq='D', include_history=False)
forecast_J_final = model_J_final.predict(future_J_final)
forecast_J_final['yhat'] = np.where(forecast_J_final['yhat'] < 0, 0, forecast_J_final['yhat'])

forecast_J_2022 = forecast_J_final.loc[
    forecast_J_final['ds'].between('2022-01-01', '2022-11-30'),
    ['ds', 'yhat']
].reset_index(drop=True)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/j31u8_r8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5qdiny0e/jqd3miy0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=79416', 'data', 'file=/tmp/tmp5qdiny0e/j31u8_r8.json', 'init=/tmp/tmp5qdiny0e/jqd3miy0.json', 'output', 'file=/tmp/tmp5qdiny0e/prophet_modelyevgb4kf/prophet_model-20250716152654.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:26:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:26:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


# total


X_2021 (2021 예측값)

In [33]:
# 지역 코드 리스트
regions = ['G', 'D', 'S', 'J']

# 결과 저장할 dict 초기화
preds_dict = {}

# 지역별 예측값 추출하여 dict에 저장
for region in regions:
    forecast = globals()[f'forecast_{region}']  # forecast_G, forecast_D, ...
    preds = forecast['yhat'].abs().values       # 음수 보정
    preds_dict[region] = preds

# DataFrame으로 변환
X_2021 = pd.DataFrame(preds_dict)
X_2021

,G,D,S,J
0,2.274850,1.936522,2.026522,1.410081
1,2.820778,2.284637,2.740350,1.534757
2,2.164533,1.608370,2.315736,0.983453
3,2.462586,2.562726,2.849365,1.249330
4,2.703262,2.966877,3.128623,1.524583
...,...,...,...,...
360,3.745360,3.728863,3.616656,1.977034
361,4.021564,4.282302,3.953699,2.299659
362,3.521430,3.752840,3.400631,1.958922
363,3.598701,3.987891,3.664922,2.101636


In [35]:
# 2021년 날짜 생성 (YYYYMMDD 형식)
date_range = pd.date_range(start='2021-01-01', end='2021-12-31', freq='D')
date_str = date_range.strftime('%Y%m%d')

# 일시 컬럼에 날짜 넣기
X_2021['일시'] = date_str.values

# 확인
print(X_2021['일시'].head())

X_2021_renamed = X_2021.rename(columns={
    'G': '광진구',
    'D': '동대문구',
    'S': '성동구',
    'J': '중랑구'})

X_2021_renamed = X_2021_renamed[['일시', '광진구', '동대문구', '성동구', '중랑구']]

print(X_2021_renamed.head())

save_path = '/content/drive/MyDrive/EURON 프로젝트/submit/X_2021.csv'

X_2021.to_csv(save_path, index=False)

print(f"✅ 저장 완료: {save_path}")

0    20210101
1    20210102
2    20210103
3    20210104
4    20210105
Name: 일시, dtype: object
         일시       광진구      동대문구       성동구       중랑구
0  20210101  2.274850  1.936522  2.026522  1.410081
1  20210102  2.820778  2.284637  2.740350  1.534757
2  20210103  2.164533  1.608370  2.315736  0.983453
3  20210104  2.462586  2.562726  2.849365  1.249330
4  20210105  2.703262  2.966877  3.128623  1.524583
✅ 저장 완료: /content/drive/MyDrive/EURON 프로젝트/submit/X_2021.csv


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# 저장할 경로 설정 (예: Google Drive 내 폴더)
save_path = '/content/drive/MyDrive/EURON 프로젝트/submit/X_2021.csv'

# 저장
X_2021.to_csv(save_path, index=False)

print(f"✅ 저장 완료: {save_path}")

✅ 저장 완료: /content/drive/MyDrive/EURON 프로젝트/submit/X_2021.csv


X_2022 (2022 예측값)

In [38]:
forecast_G_2022 = forecast_G_2022.rename(columns={'yhat': 'G'})
forecast_D_2022 = forecast_D_2022.rename(columns={'yhat': 'D'})
forecast_S_2022 = forecast_S_2022.rename(columns={'yhat': 'S'})
forecast_J_2022 = forecast_J_2022.rename(columns={'yhat': 'J'})

prophet_2022 = forecast_G_2022 \
    .merge(forecast_D_2022, on='ds') \
    .merge(forecast_S_2022, on='ds') \
    .merge(forecast_J_2022, on='ds')

prophet_2022.head()

,ds,G,D,S,J
0,2022-01-01,3.590540,2.714115,2.666828,2.044043
1,2022-01-02,3.059387,2.167246,2.441582,1.611888
2,2022-01-03,3.396250,3.461635,2.833798,1.945589
3,2022-01-04,3.352292,3.443227,2.780023,1.994527
4,2022-01-05,3.602108,3.739040,2.950794,2.173521


In [40]:
prophet_2022['일시'] = prophet_2022['ds'].dt.strftime('%Y%m%d')

prophet_2022_renamed = prophet_2022.rename(columns={
    'G': '광진구',
    'D': '동대문구',
    'S': '성동구',
    'J': '중랑구'
})

prophet_2022_renamed = prophet_2022_renamed.drop(columns=['ds'])
prophet_2022_renamed = prophet_2022_renamed[['일시', '광진구', '동대문구', '성동구', '중랑구']]

print(prophet_2022_renamed.head())

save_path = '/content/drive/MyDrive/EURON 프로젝트/submit/X_2022.csv'

# 여기를 prophet_2022_renamed로 저장해야 함
prophet_2022_renamed.to_csv(save_path, index=False)

print(f"✅ 저장 완료: {save_path}")

         일시       광진구      동대문구       성동구       중랑구
0  20220101  3.590540  2.714115  2.666828  2.044043
1  20220102  3.059387  2.167246  2.441582  1.611888
2  20220103  3.396250  3.461635  2.833798  1.945589
3  20220104  3.352292  3.443227  2.780023  1.994527
4  20220105  3.602108  3.739040  2.950794  2.173521
✅ 저장 완료: /content/drive/MyDrive/EURON 프로젝트/submit/X_2022.csv
